In [1]:
# import numpy as np
# import pandas as pd


## demo
1. 确定 判断子集 的写法
2. 确定 map的用法
3. 集合判空

In [2]:
cart = [1,2,3]
cart1 = [1,2]
cart2 = [1,2,3, 4]

carts=[cart, cart1]
print(cart in carts)
print("asdsad")
set1 = set(cart)
set2 = set([1,2])
set3 = set()
cart.append(1)

print(set2.issubset(set1))   #1

supMap = {}
supMap["c1"] = set3      #2
print(not set3)          #3
print(not set2)          #3

#print(supMap['ss'])          #4
print(cart[:-1] == cart2[:-1])

#python里不能取反
print(~False)

#不能用这种方式判断子集、、、
print([1] in [1,2,3])

print(cart)
print(cart[-2])
print(cart[-1])

True
asdsad
True
True
False
True
-1
False
[1, 2, 3, 1]
3
1


# 利用先验知识剪枝

In [3]:
def not_has_infrequent_subset(h, item, LK):
    #LK = set(LK)
#     print(LK)
    for i in range(0, h):
#         print(i)
        tmp = list(item)
        del tmp[i]
        if(tmp not in LK):
            print("以下的item不在集合中，剪枝")
            print(tmp)
            print(LK)
            return False
    return True

In [4]:
LK=[(1,2), (2,3), (1,3)]
LK2=[[1,2], [2,3]]
# print(LK2)
not_has_infrequent_subset(3, [1,2,3], LK2)

以下的item不在集合中，剪枝
[1, 3]
[[1, 2], [2, 3]]


False

## 生成下一次的频繁项集Ci+1
### link

In [5]:
#L = LMap("L"+str(i-1))
#按理说 L[k]的长度==i
#生成时 就不用item（）了，太复杂了。
def link(h, i, L):
    L.sort()
    res = []
    if( i == 0):
        for j in range(1, h+1):
            #单个数字 无法变成list。 得先变成tuple 。怪不得别人都是写的两个方法，我
            res.append(list((j,)))
    else:
#         L = LMap["L"+str(i-1)]
#         print(L)
        length = len(L)
        for j in range(length):
            for k in range(j+1, length):
#                 print(L[j][:-1])
#                 print(L[k][:-1])
                if(L[j][:-1] == L[k][:-1]):# if is Item  L[j][:-1]应该是L[j].itemSet[:-1]
                    #坑啊。。我以为不是引用复制来着 打断点太麻烦。。害得我到处print（L）
                    tmp = L[j].copy()
#                     print("i",i)
#                     print("len", L[k])
                    tmp.append(L[k][i-1])
#                     print(L)
                    if(not_has_infrequent_subset(i, tmp, L)):
                        res.append(tmp)
        
    return res

In [7]:
m = {"L2":[[1,2],[1,3]]}
L2 = [[1,2],[1,3]]
print(m)
print(link(4, 2, L2))
print(link(4, 0, L2))

{'L2': [[1, 2], [1, 3]]}
以下的item不在集合中，剪枝
[2, 3]
[[1, 2], [1, 3]]
[]
[[1], [2], [3], [4]]


## 扫描候选集Li，剪枝
### pruning

In [8]:
def pruning(sup, carts, Ci, resSet, log=False):
    res = []
    for CiItem in Ci:
        cnt = 0
        CiItem = set(CiItem)
        for cartItem in carts:
            if(CiItem.issubset(cartItem)):
                cnt += 1
        if(cnt >= sup):
            tmpList = list(CiItem)
            tmpList.sort()
            yes = Item(tmpList, cnt)
            if(log == True):
                yes.prt()
            res.append(yes.itemSet)
            resSet[yes.name] = yes
    return res
    

In [9]:
class Item:
    itemSet = []#我也不知道 为什么脑子抽了给list取名叫set
    name = ""
    sup = 0.0
    con = 0.0
    def prt(self):
        print(self.itemSet)
        print("sup is:"+str(self.sup))
        print("con is:"+str(self.con))
        
    def __init__(self, listi, sup):
        self.itemSet = listi
        
        for i in listi:
            self.name += str(i)
        self.sup = sup
        

In [10]:
cart = [1,2,3]
cartObj =Item(cart, 2)

cartObj.prt()

[1, 2, 3]
sup is:2
con is:0.0


In [11]:
def calcConf(resMap, con, height, log):
    delList = []
    for key in resMap:
        if(len(resMap[key].itemSet) >= 2):
            item = resMap[key]
            if(log):
                item.prt()
            
            name = item.name
            LastLen = len(str(item.itemSet[-1]))
            print("Name", name)
            print("item.itemSet[-1]", item.itemSet[-1])
            subName = name[:-LastLen]
            print("subName", subName)
            item.con = item.sup / resMap[subName].sup
            
    for key in resMap:
        if(len(resMap[key].itemSet) == 1 or resMap[key].con < con):
            delList.append(key)
        else:
            resMap[key].sup /= height 
    for d in delList:
        del resMap[d]

## apriori
> input: carts:购物车的集合(所有"商品"均为1~height)，support：float，confident：float
Confident（X-》Y） = S（XY）/S（X）

1. 扫描集合
    1. 如果出现次数 > support，选中。加入C1
2. 扫描集合
    1. 如果len(s)==1 || conf(x->y) > c，选中。加入L1
3. 开始用候选集Li生成下一次的频繁 i+1—— 项集集合Ci+1
    1. 如果有两集合前n-1相相等，合并他们
    2. 如果他们的所有i项的子集都在Li中，那么就放入Ci+1
4. 如果 len(Ci+1)==0，结束，否则回到1

In [12]:
def apriori(carts, height, sup, con, log = False):
    sup = sup * len(carts)
    resMap ={ } #要返回的结果 item的name -> item
    #CMap = {} # Ci -> [item, item, ……]
    LiPlus1 = []
    
    for i in range(1, height-1):  #生成i项集
        Ci = link(height, i-1, LiPlus1) 
        if(not Ci): #为空
            break
        LiPlus1 = pruning(sup, carts, Ci, resMap, log)  # 根据 i项集 剪枝生成  候选集 Li  并将sup满足的项加入resSet
    
    
    calcConf(resMap, con, height, log)
    return resMap

In [13]:
# 加载数据集
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [14]:
data = loadDataSet()
resMap = apriori(data, 5, 0.4, 0.5)

print("***********************")
for (k, v) in resMap.items():
#     print(k)
    v.prt()

Name 13
item.itemSet[-1] 3
subName 1
Name 23
item.itemSet[-1] 3
subName 2
Name 25
item.itemSet[-1] 5
subName 2
Name 35
item.itemSet[-1] 5
subName 3
Name 235
item.itemSet[-1] 5
subName 23
***********************
[1, 3]
sup is:0.4
con is:1.0
[2, 3]
sup is:0.4
con is:0.6666666666666666
[2, 5]
sup is:0.6
con is:1.0
[3, 5]
sup is:0.4
con is:0.6666666666666666
[2, 3, 5]
sup is:0.4
con is:1.0


In [15]:
data = loadDataSet()
resMap = apriori(data, 5, 0.4, 0.5)

print("***********************")
for (k, v) in resMap.items():
#     print(k)
    v.prt()

Name 13
item.itemSet[-1] 3
subName 1
Name 23
item.itemSet[-1] 3
subName 2
Name 25
item.itemSet[-1] 5
subName 2
Name 35
item.itemSet[-1] 5
subName 3
Name 235
item.itemSet[-1] 5
subName 23
***********************
[1, 3]
sup is:0.4
con is:1.0
[2, 3]
sup is:0.4
con is:0.6666666666666666
[2, 5]
sup is:0.6
con is:1.0
[3, 5]
sup is:0.4
con is:0.6666666666666666
[2, 3, 5]
sup is:0.4
con is:1.0


In [16]:
import numpy as np
import pandas as pd


In [17]:
csv = pd.read_csv("mushroom.dat",  sep=' ')
csv

,1,3,9,13,23,25,34,36,38,40,...,63,67,76,85,86,90,93,98,107,113
0,2,3,9,14,23,26,34,36,39,40,...,63,67,76,85,86,90,93,99,108,114
1,2,4,9,15,23,27,34,36,39,41,...,63,67,76,85,86,90,93,99,108,115
2,1,3,10,15,23,25,34,36,38,41,...,63,67,76,85,86,90,93,98,107,113
3,2,3,9,16,24,28,34,37,39,40,...,63,67,76,85,86,90,94,99,109,114
4,2,3,10,14,23,26,34,36,39,41,...,63,67,76,85,86,90,93,98,108,114
5,2,4,9,15,23,26,34,36,39,42,...,63,67,76,85,86,90,93,98,108,115
6,2,4,10,15,23,27,34,36,39,41,...,63,67,76,85,86,90,93,99,107,115
7,1,3,10,15,23,25,34,36,38,43,...,63,67,76,85,86,90,93,98,110,114
8,2,4,9,14,23,26,34,36,39,42,...,63,67,76,85,86,90,93,98,107,115
9,2,3,10,14,23,27,34,36,39,42,...,63,67,76,85,86,90,93,99,108,114


In [18]:
arr = csv.values # pandas 2 ndarray
max1 = []
height = arr.max()
    

In [19]:
carts = arr.tolist()

In [20]:
resMap = apriori(carts, height, 0.85, 0.8,log=True)

print("*******  *******  *********")
for (k, v) in resMap.items():
#     print(k)
    v.prt()

[34]
sup is:7913
con is:0.0
[85]
sup is:8123
con is:0.0
[86]
sup is:7923
con is:0.0
[90]
sup is:7487
con is:0.0
[34, 85]
sup is:7913
con is:0.0
[34, 86]
sup is:7905
con is:0.0
[34, 90]
sup is:7295
con is:0.0
[85, 86]
sup is:7923
con is:0.0
[85, 90]
sup is:7487
con is:0.0
[86, 90]
sup is:7287
con is:0.0
[34, 85, 86]
sup is:7905
con is:0.0
[34, 85, 90]
sup is:7295
con is:0.0
[34, 86, 90]
sup is:7287
con is:0.0
[85, 86, 90]
sup is:7287
con is:0.0
[34, 85, 86, 90]
sup is:7287
con is:0.0
[34, 85]
sup is:7913
con is:0.0
Name 3485
item.itemSet[-1] 85
subName 34
[34, 86]
sup is:7905
con is:0.0
Name 3486
item.itemSet[-1] 86
subName 34
[34, 90]
sup is:7295
con is:0.0
Name 3490
item.itemSet[-1] 90
subName 34
[85, 86]
sup is:7923
con is:0.0
Name 8586
item.itemSet[-1] 86
subName 85
[85, 90]
sup is:7487
con is:0.0
Name 8590
item.itemSet[-1] 90
subName 85
[86, 90]
sup is:7287
con is:0.0
Name 8690
item.itemSet[-1] 90
subName 86
[34, 85, 86]
sup is:7905
con is:0.0
Name 348586
item.itemSet[-1] 86
subNam

# Bug
1. <ipython-input-66-0a0d88a8b642> in calcConf(resMap, con, height, log)
     10             LastLen = len(str(item.itemSet[-1]))
     11             subName = name[:-LastLen]
---> 12             item.con = item.sup / resMap[subName].sup
     13 
     14     for key in resMap:
KeyError: ''   

> 2. 34
sup is:7913
con is:0.0
85
sup is:8123
con is:0.0
86
sup is:7923
con is:0.0
90
sup is:7487
con is:0.0
3485
sup is:7913
con is:0.0
3486
sup is:7905
con is:0.0
3490
sup is:7295
con is:0.0
8586
sup is:7923
con is:0.0
9085
sup is:7487
con is:0.0
9086
sup is:7287
con is:0.0
以下的item不在集合中，剪枝
[85, 90]
[[34, 85], [34, 86], [34, 90], [85, 86], [90, 85], [90, 86]]
以下的item不在集合中，剪枝
[86, 90]
[[34, 85], [34, 86], [34, 90], [85, 86], [90, 85], [90, 86]]
348586
sup is:7905
con is:0.0
908586
sup is:7287
con is:0.0
34
sup is:7913
怎么会有90 80 ，那不是错了吗？

# 解决
1. 加入Li,Ci前sort
2. 不能用len(resMap[key].name)>=2 <br>
len(resMap[key].itemSet 应该用这个
3. asada

## 本想着在weka里调参 跑出结果，来看是否正确，
但是weka怎么read csv文件后还需要列名呀什么的。。
尴尬

In [22]:

np.savetxt('new.csv', arr.T, delimiter = ',')